<a href="https://colab.research.google.com/github/mj00714/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Optimization attempt #3 - add a training layer


In [298]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [299]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME',], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [300]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [301]:
# check ask_amt for outliers
application_df['ASK_AMT'].describe()

count    3.429900e+04
mean     2.769199e+06
std      8.713045e+07
min      5.000000e+03
25%      5.000000e+03
50%      5.000000e+03
75%      7.742000e+03
max      8.597806e+09
Name: ASK_AMT, dtype: float64

In [302]:
ask_amt_threshold = application_df['ASK_AMT'].quantile(1-.1)
application_df = application_df[application_df['ASK_AMT'] <= ask_amt_threshold]

In [303]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     24098
T4      1485
T6      1130
T5      1057
T19     1045
T8       711
T7       698
T10      515
T13       63
T9        39
T12       18
T2         8
T15        2
Name: APPLICATION_TYPE, dtype: int64

In [304]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
threshold = 100
value_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = value_counts[value_counts < threshold].index.to_list()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

<ipython-input-304-cc414eaa0255>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")


T3       24098
T4        1485
T6        1130
T5        1057
T19       1045
T8         711
T7         698
T10        515
Other      130
Name: APPLICATION_TYPE, dtype: int64

In [305]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    15175
C2000     5521
C1200     4527
C2100     1837
C3000     1781
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 69, dtype: int64

In [306]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_value_counts = application_df['CLASSIFICATION'].value_counts()
class_value_counts_greater_than_one = class_value_counts[class_value_counts > 1]
class_value_counts_greater_than_one.head(25)

C1000    15175
C2000     5521
C1200     4527
C2100     1837
C3000     1781
C7000      688
C1700      256
C4000      172
C5000      111
C1270      103
C2700       91
C2800       80
C7100       71
C1300       55
C1280       50
C1230       36
C1400       34
C1240       29
C2300       29
C7200       27
C7120       15
C8000       15
C6000       14
C1250       14
C1500       12
Name: CLASSIFICATION, dtype: int64

In [307]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_threshold = 1000
class_value_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = class_value_counts[class_value_counts < class_threshold].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

<ipython-input-307-be3e9e4a1dc5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


C1000    15175
C2000     5521
C1200     4527
Other     2028
C2100     1837
C3000     1781
Name: CLASSIFICATION, dtype: int64

In [308]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df)
application_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [309]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df['IS_SUCCESSFUL']
X = application_dummies_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25)

In [310]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [311]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 16
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 4
hidden_nodes_layer4 = 2
hidden_nodes_layer5 = 1
hidden_nodes_layer6 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# # Fifth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))

# # Sixth hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_84 (Dense)            (None, 16)                704       
                                                                 
 dense_85 (Dense)            (None, 8)                 136       
                                                                 
 dense_86 (Dense)            (None, 4)                 36        
                                                                 
 dense_87 (Dense)            (None, 2)                 10        
                                                                 
 dense_88 (Dense)            (None, 1)                 3         
                                                                 
Total params: 889 (3.47 KB)
Trainable params: 889 (3.47 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [312]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [313]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
242/242 [==============================] - 2s 2ms/step - loss: 2.5093 - accuracy: 0.5988
Epoch 2/50
242/242 [==============================] - 0s 2ms/step - loss: 1.1300 - accuracy: 0.6764
Epoch 3/50
242/242 [==============================] - 1s 4ms/step - loss: 0.9626 - accuracy: 0.6877
Epoch 4/50
242/242 [==============================] - 1s 3ms/step - loss: 0.9258 - accuracy: 0.6969
Epoch 5/50
242/242 [==============================] - 1s 3ms/step - loss: 0.6884 - accuracy: 0.7029
Epoch 6/50
242/242 [==============================] - 1s 3ms/step - loss: 0.6297 - accuracy: 0.7215
Epoch 7/50
242/242 [==============================] - 1s 2ms/step - loss: 0.6010 - accuracy: 0.7162
Epoch 8/50
242/242 [==============================] - 0s 2ms/step - loss: 0.5935 - accuracy: 0.7271
Epoch 9/50
242/242 [==============================] - 1s 2ms/step - loss: 0.5729 - accuracy: 0.7293
Epoch 10/50
242/242 [==============================] - 0s 2ms/step - loss: 0.5685 - accuracy: 0.7309

In [314]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

724/724 - 1s - loss: 0.5469 - accuracy: 0.7389 - 1s/epoch - 2ms/step
Loss: 0.5469141602516174, Accuracy: 0.7389426231384277


In [315]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
